In [1]:
class node:
    """
    Construct an double linked observer tree by nodes    
    """
    def __init__(self, left=None,right=None,op='atomic',interval=[],depth=0,atomic=None):
        """   
        args:
            left(node): left child node
            right(node): right child node
            op(str):operator type
            interval(list): interval of the opeartor
        """
        self.left = left
        self.right = right
        self.op = op
        self.interval = interval
        self.depth = depth
        self.atomic = atomic
        self.parents = set()
        
    def add_parent(self, node):
        self.parents.add(node)

In [2]:
def MakeTree(file):
    """
    Construct the big observer tree from assembly file
    args:
        assembly file name
    return:
        root node
    """
    Opnum = lambda word: word[0]+str(int(word[1:]))# remove redundant '0's, e.g. convert 's000' -> 's0'
    name2node,node2name,lineCount = {},{},0
    with open(file) as f:
        lines = f.readlines()
        for line in lines:
            words = line.strip().lower().split()        
            if(len(words)==0 or words[0][0]=='#'):# skip the empty line and commented line
                continue
            for i,word in enumerate(words[1:]):
                if(not word.isdigit()):
                    words[i+1]=Opnum(word)
            print(words)
            if(words[0]=='load' or words[0]=='load_ft'):
#                 if(words[1] not in name2node):
#                     name2node[words[1]] = node(atomic=words[1])
#                     node2name[name2node[words[1]]] = words[1]
#                 curNode = node(left=name2node[words[1]],op='load',depth=name2node[words[1]].depth+1)# count from 1
                curNode = node(op='load',atomic=words[1])# count from 1
                
            elif(words[0]=='not'):
                curNode = node(left=name2node[words[1]],op=words[0],interval=[int(words[2])],depth=name2node[words[1]].depth+1)
            elif(words[0]=='boxbox'):
                curNode = node(left=name2node[words[1]],op=words[0],interval=[int(words[2])],depth=name2node[words[1]].depth+1)
            elif(words[0]=='boxdot'):
                curNode = node(left=name2node[words[1]],op=words[0],interval=[int(words[2]),int(words[3])],depth=name2node[words[1]].depth+1)
            elif(words[0]=='and' or words[0]=='until'):
                curNode = node(left=name2node[words[1]],right=name2node[words[2]],op=words[0],depth=max(name2node[words[1]].depth,name2node[words[2]].depth)+1)
                name2node[words[2]].add_parent(curNode)# assign parent node
            elif(words[0]=='end'):
                curNode = node(left=name2node[words[1]],op=words[0],depth=name2node[words[1]].depth+1)                
            name2node['s'+str(lineCount)] = curNode
            if(words[1] in name2node):
                name2node[words[1]].add_parent(curNode)# assign parent node
            node2name[curNode] = 's'+str(lineCount)
            lineCount+=1
    return curNode,name2node,node2name

In [3]:
file = "assembly.txt"
root,name2node,node2name = MakeTree(file)
cutSet={'s2','s4'}# user defined cut
#cutSet={'s0'}
cutNodeSet = {name2node[word] for word in cutSet}# map cutSet to node

class group:
    class miniGroup:
        def __init__(self):
            self.member = set()
        def add(self,node):
            self.member.add(node)
        def update(self, mini_group):
            self.member.update(mini_group.member)
    
    def __init__(self):
        self.tagUsed = 0
        self.tag2group = {}
        self.node2tag = {}
    
    def getNodeTag(self, node):
        return self.node2tag[node]
    
    def getNewTag(self):
        # construct a new tag label and map the new tag to a new miniGroup
        self.tagUsed += 1
        self.tag2group[self.tagUsed] = group.miniGroup()
        return self.tagUsed
    
    def add(self, node, tag):
        # map a node to the tag
        self.node2tag[node] = tag
        if(tag not in self.tag2group):
            self.tag2group[tag] = group.miniGroup()
        self.tag2group[tag].add(node)
    
    def combine_group(self, tag1, tag2):
        # comine two groups (merge tag2 group into tag1 group)
        miniGroup1 = self.tag2group[tag1]
        miniGroup2 = self.tag2group[tag2]
        if not(miniGroup1 is miniGroup2): # test if the two references point to the same miniGroup
            miniGroup1.update(miniGroup2)
            self.tag2group[tag2] = miniGroup1
        
bigGroup = group()

# recursive programming
# DFS the observer tree to do partition
def assignGroup(root,setTag):
    if(root):
        if(root in bigGroup.node2tag):
            oldTag = bigGroup.getNodeTag(root)
            bigGroup.combine_group(oldTag,setTag)
        else:
            #if((root in cutNodeSet) or (len(root.parents)==0)):# initialize a new partition
            if((root in cutNodeSet) or (root.op=='end')):# initialize a new partition
                setTag = bigGroup.getNewTag()
            bigGroup.add(root,setTag)
            assignGroup(root.left,setTag)
            assignGroup(root.right,setTag)
                   
assignGroup(root,0)

['load', 'a0']
['load', 'a1']
['and', 's0', 's1']
['load', 'a0']
['boxdot', 's3', '3', '5']
['and', 's2', 's4']
['end', 's5']


In [4]:
# map the group tag to the list of nodes in the group
group2nodes={}
# DFS to get all the node and its group
def DFS(root):
    if(root):
        certainGroup = bigGroup.tag2group[bigGroup.node2tag[root]]
        if(certainGroup not in group2nodes):
            group2nodes[certainGroup] = []    
        group2nodes[certainGroup].append(root)
        DFS(root.left)
        DFS(root.right)

DFS(root)

for keys in group2nodes:
    print('--------------------')
    for nodes in group2nodes[keys]:
        print(nodes.op)
print('--------------------')


--------------------
end
and
--------------------
and
load
load
--------------------
boxdot
load
--------------------


In [5]:
def topologicalSort(output,group,group2node, cutNodeSet):
    '''
    do topological sort of the tree branch
    '''
    print(output)
    visited = set()
    stack = []
    for nodes in group2node[group]:
        if(nodes not in visited):
            topSortUntil(nodes, visited, stack, group2node[group], cutNodeSet)
    stack.reverse()
    for e in stack:
        print(e.op)
    print()
    return stack
    
def topSortUntil(nodes, visited, stack, boundedGroup, cutNodeSet):
    visited.add(nodes)
    if(len(nodes.parents)==0 or nodes in cutNodeSet):
        stack.append(nodes)
    else:
        for parent in nodes.parents:
            if(parent not in visited and parent in boundedGroup):
                topSortUntil(parent,visited, stack, boundedGroup,cutNodeSet)
        stack.append(nodes)

In [6]:
outputFile = lambda i: 'core_'+str(i)
node2core = {}
sortResult = {}

#sortResult = {i:topologicalSort(outputFile(i),group,group2nodes,cutNodeSet) for i,group in enumerate(group2nodes)}
for i,group in enumerate(group2nodes):
    for nodes in group2nodes[group]:
        node2core[nodes] = i
    sortResult[i] = topologicalSort(outputFile(i),group,group2nodes,cutNodeSet) 

core_0
and
end

core_1
load
load
and

core_2
load
boxdot



In [7]:
# Generate output
for i in range(len(sortResult)):
    lineCount = 0
    prog = []
    node2line = {}
    for results in sortResult[i]: # traverse the stack of each core
        if results.left and results.left not in sortResult[i]: # fetch result from another core, add extra 'load' operation
            result_left = node(op='loadc',atomic='c'+str(node2core[results.left])+'_'+node2name[results.left])
            result_left.add_parent(results)
            node2core[result_left] = i
            node2line[result_left] = lineCount
            prog.append('loadc '+'c'+str(node2core[results.left])+'_'+node2name[results.left])
            results.left = result_left # establish new connection
            lineCount += 1
        if results.right: # and/until operation
            if results.right not in sortResult[i]: # fetch result from another core
                result_right = node(op='loadc',atomic='c'+str(node2core[results.right])+'_'+node2name[results.right])
                result_right.add_parent(results)
                node2core[result_right] = i
                node2line[result_right] = lineCount
                prog.append('loadc '+'c'+str(node2core[results.right])+'_'+node2name[results.right])
                results.right = result_right # establish new connection
                lineCount += 1
        node2line[results] = lineCount
        if results.op == 'load':
            prog.append(results.op+' '+results.atomic)        
        elif results.op == 'not' or results.op == 'end':
            prog.append(results.op+' s'+str(node2line[results.left]))
        elif results.op == 'boxbox':
            prog.append(results.op+' s'+str(node2line[results.left])+' '+str(results.interval[0]))
        elif results.op == 'boxdot':
            prog.append(results.op+' s'+str(node2line[results.left])+' '+str(results.interval[0])+' '+str(results.interval[1]))
        elif results.op == 'and' or results.op == 'until':
            prog.append(results.op+' s'+str(node2line[results.left])+' s'+str(node2line[results.right]))  
        lineCount += 1
        
        for parent in results.parents:
            if parent not in sortResult[i]: # the result will be sent to other core
                prog.append('endc'+' s'+str(lineCount-1)+' c'+str(node2core[parent])+'_'+node2name[results])
                lineCount += 1

    print('Command On Processing Core_',i,':')
    for x in prog:
        print(x)
    print('---------------------------------')

Command On Processing Core_ 0 :
loadc c1_s2
loadc c2_s4
and s0 s1
end s2
---------------------------------
Command On Processing Core_ 1 :
load a1
load a0
and s1 s0
endc s2 c0_s2
---------------------------------
Command On Processing Core_ 2 :
load a0
boxdot s0 3 5
endc s1 c0_s4
---------------------------------
